In [ ]:
import catllm as cat
import pandas as pd
import os

Here we set our API's. I'm reading from a file on my machine, but you can also just copy and paste the string.

In [ ]:
# Replace these with your actual API keys
anthropic_api_key = "ANTHROPIC_API_KEY"
openai_api_key = "OPENAI_API_KEY"
perplexity_api_key = "PERPLEXITY_API_KEY"
mistral_api_key = "MISTRAL_API_KEY"
google_api_key = "GOOGLE_API_KEY"
huggingface_api_key = "HUGGINGFACE_API_KEY"
xai_api_key = "XAI_API_KEY"

Generating some fake data and categories to assign to it.

In [ ]:
#data we will be categoizing
list_names = ["becuase i dont like living here", "for a bigger house", "to be with my wife"]

#categories we want to extract from the data
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard"]

Below we're running the most simple example of the package. The essential elements are defined above, which are 
1. The survey input you want to categorize (a column of text data)
2. The categories you want to extract (defined by you)

In [ ]:
test_anthropic = cat.multi_class(
    survey_input= list_names, 
    user_model="claude-sonnet-4-20250514",
    categories=user_categories,
    api_key=anthropic_api_key)

test_anthropic.head()

The package will automatically detect when you switch model source. Below, we're running an example with GPT-5. Just remember to make sure you input the correct API!

In [ ]:
test_openai = cat.multi_class(
    survey_question="why did you move?", 
    survey_input= list_names, 
    user_model="gpt-5",
    categories=user_categories,
    api_key=openai_api_key)

test_openai.head()

One of the most powerful features of CatLLM is the ability to call on the thousands of models available on Huggingface. Here, you might need to be more explicit about where the model is coming.

Here, we set model_source to "Huggingface" so that CatLLM knows where to pull from. Again, make sure you have the correct API key! 

WARNING: Using Huggingface without a Hugginface pro subscription is not recommended. The free tier will only allow you to cycle through a few rows. 

In [ ]:
test_huggingface = cat.multi_class(
    survey_input= list_names, 
    user_model="meta-llama/Llama-4-Maverick-17B-128E-Instruct:groq",
    model_source="Huggingface",
    categories=user_categories,
    api_key=huggingface_api_key)

test_huggingface.head()

Let's add a bit more complexity for better results. 

Rather than just giving the model the task of categorizing without any context, let's provide CatLLM with the survey question that was asked of the respondent. 

Let's also ask CatLLM to provide a bit more context on what its role and goal is. 

Keep in mind that these features make for a longer prompt, which translates to higher costs from the model provider. Only use this feature if you want to improve the quality of your output and you're willing to pay a bit more.

In [ ]:
test_anthropic_with_context = cat.multi_class(
    survey_input= list_names, 
    survey_question = "Why did you move?", # add your survey question here
    context_prompt = True, # ask Cat-LLM to provide a bit more context on the cateorization task
    user_model="claude-sonnet-4-20250514",
    categories=user_categories,
    api_key=anthropic_api_key)

test_anthropic_with_context.head()

Now, let's add even more complexity. Before categorizing, let's ask the model to take a "step back" and observe the bigger picture here. 

In this case, we will ask the model to consider broader reasons for why people move so that we can get it to reason towards a better answer. 

Again, this will increase your prompt size, so use only if you don't mind the additional API costs.

In [ ]:
test_anthropic_step_back = cat.multi_class(
    survey_input= list_names, 
    survey_question = "Why did you move?",
    context_prompt = True,
    step_back_prompt = True, # ask the model to consider the broader conceptual background
    user_model="claude-sonnet-4-20250514",
    categories=user_categories,
    api_key=anthropic_api_key)

test_anthropic_step_back.head()

The step back method works to improve the quality of your categorizations, but chain of verification (CoVe) should improve them even further. 

What is CoVe? Here, we use multiple prompts to get the model to "reason" through its response by considering more than surface level information. That is, we ask the model a series of "verification" questions to get the model to think a bit more about its initial answer and have the opportunity to revise it.

WARNING: Although we can combine all of these features, doing so will multiply your costs. Only combine all features if you're hoping for the absolute best output and don't mind paying 10 times as much for a small improvement.

In [ ]:
test_anthropic_cove = cat.multi_class(
    survey_input= list_names, 
    survey_question = "Why did you move?",
    context_prompt = True,
    step_back_prompt = True,
    chain_of_verification = True, # allow the model to think through its initial response and change its answer
    user_model="claude-sonnet-4-20250514",
    categories=user_categories,
    api_key=anthropic_api_key)

test_anthropic_cove.head()

## Chain-of-Thought (CoT) - Now the Default

Sometimes, a simpler prompt might be able to get us all the way there. Chain-of-Thought (CoT) can improve results without needing multiple prompts and increases costs at a fraction of the price of CoVe.

CoT essentially asks the model to reason through its response a bit more before outputting a response. It does so within the same prompt, rather than needing many prompts.

**Note:** As of the latest version, `chain_of_thought=True` is now the **default setting**. This means all the examples above were already using CoT! If you want to disable it (for faster/cheaper processing), you can set `chain_of_thought=False`.

Below is an example where we explicitly disable CoT to show the difference:

In [ ]:
# Example with CoT explicitly DISABLED to show the difference
test_anthropic_no_cot = cat.multi_class(
    survey_input=list_names, 
    survey_question="Why did you move?",
    chain_of_thought=False,  # Disable CoT for faster/cheaper processing
    user_model="claude-sonnet-4-20250514",
    categories=user_categories,
    api_key=anthropic_api_key)

test_anthropic_no_cot.head()

Lastly, CatLLM can also generate categories for you by extracting the most common categories from your column of survey responses. 

Here, we set categories to "auto" and let the model identify the most important/common categories present in your data.

In [ ]:
data_path = '/Users/chrissoria/Documents/Research/empirical_investigation_llm/data/processed/'
UCNets_Survey_Data = pd.read_csv(data_path+"unique_main_a19i.csv")

test_anthropic_auto_categories = cat.multi_class(
    survey_input= UCNets_Survey_Data['Response'], 
    survey_question = "Why did you move?",
    user_model="claude-sonnet-4-20250514",
    categories="auto",
    api_key=anthropic_api_key)

test_anthropic_auto_categories.head()

## Safety Feature: Incremental Saves

When processing large datasets, API calls can fail midway through. The `safety` feature saves your progress after each API call, so you don't lose your work if something goes wrong.

**How it works:**
- Set `safety=True` and provide a `filename`
- CatLLM will save a CSV file after processing each row
- If the process fails, you can resume from where you left off

This is especially useful for:
- Large datasets (hundreds or thousands of rows)
- Expensive API calls (CoVe, step-back prompting)
- Unstable network connections

In [ ]:
# Example with safety feature enabled
test_with_safety = cat.multi_class(
    survey_input=list_names, 
    survey_question="Why did you move?",
    user_model="gpt-4o",
    categories=user_categories,
    safety=True,  # Enable incremental saves
    filename="categorized_results.csv",  # Required when safety=True
    save_directory="./output",  # Optional: specify output directory
    api_key=openai_api_key)

test_with_safety.head()

## Google Models with Extended Thinking (thinking_budget)

Google's Gemini models support an extended thinking feature through the `thinking_budget` parameter. This allows the model to "think longer" before responding, potentially improving accuracy for complex categorization tasks.

**Note:** This feature is only available for Google models and requires a Google AI Studio API key. The free tier has severe rate limits (5 requests per minute), so a billing account is recommended for large-scale use.

In [ ]:
# Example with Google Gemini and extended thinking
test_google_thinking = cat.multi_class(
    survey_input=list_names, 
    survey_question="Why did you move?",
    user_model="gemini-2.0-flash",  # Google Gemini model
    categories=user_categories,
    thinking_budget=1024,  # Allow extended thinking (tokens for reasoning)
    api_key=google_api_key)

test_google_thinking.head()

## Few-Shot Learning with Examples

You can provide up to 6 examples to help guide the model's categorization. This is especially useful when:
- Your categories have nuanced definitions
- You want consistent interpretation across responses
- The model is making systematic errors

Each example should show how a response maps to categories.

In [ ]:
# Example with few-shot learning
test_with_examples = cat.multi_class(
    survey_input=list_names, 
    survey_question="Why did you move?",
    user_model="gpt-4o",
    categories=user_categories,
    # Provide examples to guide categorization
    example1="'I moved to be closer to my girlfriend' -> category 1 (partner/spouse)",
    example2="'Got a new job in Seattle' -> category 3 (job/career change)",
    example3="'Rent was too expensive' -> category 5 (financial reasons)",
    example4="'Wanted a bigger backyard for the kids' -> category 6 (home features)",
    api_key=openai_api_key)

test_with_examples.head()

---

# Image Classification with image_multi_class

CatLLM also supports multi-label image classification. The `image_multi_class` function works similarly to `multi_class` but processes images instead of text.

**Key features:**
- Accepts a list of image file paths or a folder path
- Supports the same prompt strategies (CoT, CoVe, step-back, context)
- Works with vision-capable models from OpenAI, Anthropic, Google, and Mistral

In [ ]:
# Define image categories
image_categories = [
    "contains a person",
    "outdoor scene",
    "contains text or writing",
    "has animals",
    "is a photograph (not illustration)"
]

# List of image paths (replace with your actual paths)
image_paths = [
    "path/to/image1.jpg",
    "path/to/image2.png",
    "path/to/image3.jpeg"
]

# Basic image classification
image_results = cat.image_multi_class(
    image_description="Various photographs and images",
    image_input=image_paths,  # Can also pass a folder path
    categories=image_categories,
    user_model="gpt-4o",
    api_key=openai_api_key)

image_results.head()

## Image Classification with Advanced Prompt Strategies

Just like text classification, you can use all the prompt enhancement strategies with images:

- **chain_of_thought** (default=True): Step-by-step visual analysis
- **chain_of_verification**: Re-examines the image to verify categorization
- **step_back_prompt**: Analyzes key visual features before classification
- **context_prompt**: Adds expert visual analyst role

In [ ]:
# Image classification with Chain of Verification
# The model will re-examine the image to verify its initial categorization
image_results_cove = cat.image_multi_class(
    image_description="Various photographs and images",
    image_input=image_paths,
    categories=image_categories,
    user_model="gpt-4o",
    chain_of_verification=True,  # Re-examine image to verify
    context_prompt=True,  # Add expert analyst role
    api_key=openai_api_key)

image_results_cove.head()

## Image Classification from a Folder

Instead of listing individual file paths, you can pass a folder path and CatLLM will automatically find all supported image files (PNG, JPG, JPEG, GIF, WebP, etc.).

In [ ]:
# Classify all images in a folder
folder_path = "path/to/your/images/folder"

image_results_folder = cat.image_multi_class(
    image_description="Collection of images to categorize",
    image_input=folder_path,  # Pass folder path instead of list
    categories=image_categories,
    user_model="claude-sonnet-4-20250514",  # Works with Anthropic too
    safety=True,  # Save progress incrementally
    filename="image_classification_results.csv",
    api_key=anthropic_api_key)

image_results_folder.head()

## Summary: Prompt Strategy Comparison

| Strategy | Cost Impact | When to Use |
|----------|-------------|-------------|
| `chain_of_thought=True` (default) | Low (+10-20%) | Always - provides step-by-step reasoning |
| `context_prompt=True` | Low (+5-10%) | When you want expert-level analysis |
| `step_back_prompt=True` | Medium (+30-50%) | Complex categorization with subtle distinctions |
| `chain_of_verification=True` | High (+300-500%) | When accuracy is critical, budget allows |
| `thinking_budget` (Google only) | Medium-High | Complex reasoning tasks with Gemini models |

**Recommendation:** Start with the defaults (`chain_of_thought=True`) and add more strategies only if you need improved accuracy and can afford the additional API costs.